In [27]:
###############################################################
# LIMPIEZA ROBUSTA DE BASE DE DATOS
# Archivo: copia.xlsx
# Autor: Adylene Baylon
# Descripción: Limpieza avanzada y preparación de datos para análisis.
###############################################################

import pandas as pd
import numpy as np
import unicodedata

# ==========================
# 1️⃣ CARGA Y EXPLORACIÓN
# ==========================
df = pd.read_excel("copia.xlsx")

print("Dimensiones originales:", df.shape)
print("\nColumnas originales:\n", list(df.columns))

# ==========================
# 2️⃣ NORMALIZACIÓN DE NOMBRES DE COLUMNAS
# ==========================
def normalizar_texto(texto):
    if not isinstance(texto, str):
        texto = str(texto)
    texto = texto.strip().lower()
    texto = ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )
    texto = texto.replace(' ', '_').replace('-', '_')
    return texto

df.columns = [normalizar_texto(col) for col in df.columns]

# ==========================
# 3️⃣ ELIMINAR DUPLICADOS Y COLUMNAS VACÍAS
# ==========================
df = df.drop_duplicates()
df = df.dropna(axis=1, how='all')  # elimina columnas totalmente vacías

# ==========================
# 4️⃣ DETECCIÓN Y CONVERSIÓN AUTOMÁTICA DE TIPOS DE DATOS
# ==========================
for col in df.columns:
    # Intentar convertir columnas a numéricas si es posible
    try:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    except:
        pass

# ==========================
# 5️⃣ LIMPIEZA Y RELLENO DE NULOS
# ==========================
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        # Si la columna es numérica
        df[col] = df[col].fillna(0)
        # Convertir a int si no hay decimales significativos
        if (df[col] % 1 == 0).all():
            df[col] = df[col].astype(int)
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        # Si es fecha → rellenar con la moda o fecha más común
        moda_fecha = df[col].mode()
        if not moda_fecha.empty:
            df[col] = df[col].fillna(moda_fecha[0])
    else:
        # Si es texto → rellenar con "Desconocido"
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace(['nan', 'none', '', 'na'], 'desconocido')
        df[col] = df[col].fillna('desconocido')

# ==========================
# 6️⃣ DETECCIÓN Y CORRECCIÓN DE FECHAS
# ==========================
for col in df.columns:
    if "fecha" in col or "date" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        # Rellenar nulos con la fecha más frecuente si existe
        if df[col].notna().any():
            df[col] = df[col].fillna(df[col].mode()[0])

# ==========================
# 7️⃣ CONTROL DE OUTLIERS (EDAD / AÑO)
# ==========================
if 'edad' in df.columns:
    df = df[(df['edad'] >= 0) & (df['edad'] <= 120)]

if 'anio' in df.columns:
    df = df[(df['anio'] >= 2000) & (df['anio'] <= 2025)]

# ==========================
# 8️⃣ OPTIMIZACIÓN DE MEMORIA
# ==========================
for col in df.select_dtypes(include=['int', 'float']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# ==========================
# 9️⃣ REVISIÓN FINAL
# ==========================
print("\nDimensiones finales:", df.shape)
print("\nValores nulos por columna:\n", df.isnull().sum())
print("\nTipos finales de datos:\n", df.dtypes)

# ==========================
# 🔟 EXPORTAR RESULTADO FINAL
# ==========================
df.to_excel("casos.xlsx", index=False)
print("\n✅ Base de datos limpia guardada como 'copia_limpia_final.xlsx'")


Dimensiones originales: (7328, 80)

Columnas originales:
 ['euv', 'euv_caso', 'victima_sexo', 'victima_nacionalidad', 'victima_fecha_nacimiento', 'victima_edad', 'victima_estado_civil', 'victima_escolaridad', 'victima_ingreso_economico', 'victima_ocupacion', 'victima_pueblo_indigena', 'victima_migrante', 'victima_afrodescendiente', 'victima_orientacion_sexual', 'victima_identidad_genero', 'victima_vive_extranjero', 'victima_colonia', 'victima_localidad', 'victima_municipio', 'victima_entidad', 'victima_discapacidad', 'victima_discapacidad_por_violencia', 'victima_enfermedad_cronica', 'victima_padecimiento_psiquiatrico', 'victima_tipo_tratamiento', 'victima_embarazada', 'victima_caso_name', 'victima_servicio_medico', 'victima_situacion_calle', 'victima_adiccion', 'victima_dependientes', 'total_hijo_a', 'total_madre_padre', 'total_dependientes_otros', 'hecho_lugar_hechos', 'hecho_lugar_cual', 'hecho_domicilio_victima', 'hecho_extranjero_victima', 'hecho_localidad', 'hecho_municipio', 'he

C:\Users\ady_b\AppData\Local\Temp\ipykernel_15904\2905225652.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')



Dimensiones finales: (7323, 79)

Valores nulos por columna:
 euv                           0
euv_caso                      0
victima_sexo                  0
victima_nacionalidad          0
victima_fecha_nacimiento      0
                             ..
usuario_hecho_entidad         0
usuario_hecho_municipio       0
usuario_hecho_dependencia     0
usuario_hecho_cargo           0
usuario_hecho_prog_federal    0
Length: 79, dtype: int64

Tipos finales de datos:
 euv                                   object
euv_caso                              object
victima_sexo                          object
victima_nacionalidad                  object
victima_fecha_nacimiento      datetime64[ns]
                                   ...      
usuario_hecho_entidad                 object
usuario_hecho_municipio               object
usuario_hecho_dependencia             object
usuario_hecho_cargo                   object
usuario_hecho_prog_federal            object
Length: 79, dtype: object

✅ Base de dat

In [28]:
import pandas as pd
import numpy as np
import unicodedata

# ==========================
# 1️⃣ CARGA Y EXPLORACIÓN
# ==========================
df = pd.read_excel("casos.xlsx")

#Eliminar columnas con aportación mínima
del df["victima_migrante"]
del df["victima_afrodescendiente"]
del df["victima_identidad_genero"]
del df["victima_vive_extranjero"]
del df["victima_colonia"]
del df["victima_localidad"]
del df["victima_discapacidad_por_violencia"]
del df["victima_enfermedad_cronica"]
del df["victima_padecimiento_psiquiatrico"]
del df["victima_tipo_tratamiento"]
del df["victima_embarazada"]
del df["victima_caso_name"]
del df["victima_situacion_calle"]
del df["victima_dependientes"]
del df["total_madre_padre"]
del df["total_dependientes_otros"]
del df["hecho_extranjero_victima"]
del df["hecho_colonia"]
del df["detalle_violencia_otro"]
del df["tipo_agresor_sexual"]
del df["acoso_hostigamiento"]
del df["hechos_culmino_muerte"]
del df["tipo_muerte"]
del df["aborto"]
#del df["quemadura_corrosion"]
#del df["usuario_hecho_entidad"]
#del df["deleted_at"]


# Después de la limpieza
num_cols_despues = df.shape[1]
print(f"Número de columnas después de limpieza: {num_cols_despues}")

# ==========================
# 🔟 EXPORTAR RESULTADO FINAL
# ==========================
df.to_excel("casos3.xlsx", index=False)
print("\n✅ Base de datos limpia guardada como 'copia_limpia_final.xlsx'")

Número de columnas después de limpieza: 55

✅ Base de datos limpia guardada como 'copia_limpia_final.xlsx'


In [17]:
# Eliminar filas con más del 60% de "desconocido" o nulos
umbral = 0.75
df = df[df.apply(lambda fila: (fila == 'desconocido').mean() < umbral, axis=1)]

# ==========================
# 🔟 EXPORTAR RESULTADO FINAL
# ==========================
df.to_excel("casos2.xlsx", index=False)
print("\n✅ Base de datos limpia guardada como 'copia_limpia_final.xlsx'")


✅ Base de datos limpia guardada como 'copia_limpia_final.xlsx'


In [26]:
import pandas as pd

# Carga la base
df = pd.read_excel("copia.xlsx")

# Número de registros
num_registros = len(df)
print("Número de registros:", num_registros)


Número de registros: 7328


**REGISTROS**
calcula el tipo de dato y la fecuencia de cada uno

In [43]:
import pandas as pd

# Cargar la base de datos
df = pd.read_excel("casos2.xlsx")

# Para una columna específica
frecuencia_columna = df['victima_edad'].value_counts()
print(frecuencia_columna)

tipo = df["victima_edad"].dtype

tipo



victima_edad
0     1520
29     203
31     200
35     186
26     185
      ... 
87       1
85       1
82       1
81       1
95       1
Name: count, Length: 90, dtype: int64


dtype('int64')

In [36]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel("casos3.xlsx")

# Eliminar filas donde edad == 0 y estado_civil != "desconocido"
df_filtrado = df[~((df["victima_edad"] == 0) & (df["victima_estado_civil"].str.lower() != "desconocido"))]

# Guardar el resultado en un nuevo archivo
df_filtrado.to_excel("casos2.xlsx", index=False)
